# 第2回講義 宿題

## 課題
今回のLessonで学んだことを元に，MNISTのファッション版 (Fashion MNIST，クラス数10) をソフトマックス回帰によって分類してみましょう．

Fashion MNISTの詳細については以下のリンクを参考にしてください．

Fashion MNIST: https://github.com/zalandoresearch/fashion-mnist

### 目標値
Accuracy: 80%

### ルール
- **下のセルで指定されている`x_train、y_train`以外の学習データは使わないでください．**
- **ソフトマックス回帰のアルゴリズム部分の実装はnumpyのみで行ってください** (sklearnやtensorflowなどは使用しないでください)．
    - データの前処理部分でsklearnの関数を使う (例えば `sklearn.model_selection.train_test_split`) のは問題ありません．

### 提出方法
- 2つのファイルを提出していただきます．
    1. テストデータ (`x_test`) に対する予測ラベルを`submission_pred.csv`として保存し，**Homeworkタブから`lecture02`を選択して**提出してください．
    2. それに対応するpythonのコードを`submission_code.py`として保存し，**Homeworkタブから`lecture02 (code)`を選択して**提出してください．
      
- なお，採点は1で行い，2はコードの確認用として利用します（成績優秀者はコード内容を公開させていただくかもしれません）．コードの内容を変更した場合は，**1と2の両方を提出し直してください**．

### 評価方法
- 予測ラベルの`y_test`に対する精度 (Accuracy) で評価します．
- 即時採点しLeader Boardを更新します．（採点スケジュールは別アナウンス）
- 締切後の点数を最終的な評価とします．

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### データの読み込み（このセルは修正しないでください）

In [2]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/x_train.npy')
    y_train = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('drive/MyDrive/Colab Notebooks/DLBasics2023_colab/Lecture02/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

In [3]:
import os
import sys

import numpy as np
import pandas as pd

sys.modules['tensorflow'] = None

def load_fashionmnist():
    # 学習データ
    x_train = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture02/data/x_train.npy')
    y_train = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture02/data/y_train.npy')
    
    # テストデータ
    x_test = np.load('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture02/data/x_test.npy')
    
    x_train = x_train.reshape(-1, 784).astype('float32') / 255
    y_train = np.eye(10)[y_train.astype('int32')]
    x_test = x_test.reshape(-1, 784).astype('float32') / 255
    
    return x_train, y_train, x_test

### ソフトマックス回帰の実装

In [4]:
x_train, y_train, x_test = load_fashionmnist()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def np_log(x):
    return np.log(np.clip(a=x, a_min=1e-10, a_max=1e+10))

def softmax(x, axis=1):
    x -= x.max(axis, keepdims=True) # expのoverflowを防ぐ # WRITE ME
    x_exp = np.exp(x)
    return x_exp / x_exp.sum(axis, keepdims=True) # WRITE ME

def train(x, t, eps=1.0):
  global W, b
  batch_size = x.shape[0]

  t_hat = softmax(np.matmul(x,W) + b)

  cost = ((- t * np_log(t_hat)).sum(axis=1).mean())
  
  delta = t_hat - t
  dW = np.matmul(x.T, delta) / batch_size # shape: (入力の次元数, 出力の次元数)
  db = np.matmul(np.ones(shape=(batch_size,)), delta) / batch_size # shape: (出力の次元数,)
  W -= eps * dW
  b -= eps * db

  return cost
  # WRITE ME

def valid(x, t):
  t_hat = softmax(np.matmul(x,W) + b)
  cost = ((- t * np_log(t_hat)).sum(axis=1).mean())
  return cost, t_hat

# weights
W = np.random.uniform(low=-0.08, high=0.08, size=(784, 10)).astype('float32')# WRITE ME
b = np.zeros(shape=(10,)).astype('float32')# WRITE ME

# 学習データと検証データに分割
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1)

for epoch in tqdm(range(5000)):
    # オンライン学習
    cost = train(x_train, y_train, 0.3)
    cost, y_pred_valid = valid(x_valid, y_valid)
    if epoch % 10== 9 or epoch == 0:
        print('EPOCH: {}, Valid Cost: {:.3f}, Valid Accuracy: {:.3f}'.format(
            epoch + 1,
            cost,
            accuracy_score(y_valid.argmax(axis=1), y_pred_valid.argmax(axis=1))
        ))
    # WRITE ME


  0%|          | 1/5000 [00:00<45:16,  1.84it/s]

EPOCH: 1, Valid Cost: 2.112, Valid Accuracy: 0.332


  0%|          | 10/5000 [00:04<39:03,  2.13it/s]

EPOCH: 10, Valid Cost: 2.131, Valid Accuracy: 0.539


  0%|          | 20/5000 [00:10<50:10,  1.65it/s]

...

EPOCH: 4980, Valid Cost: 0.509, Valid Accuracy: 0.843


100%|█████████▉| 4990/5000 [43:07<00:04,  2.16it/s]

EPOCH: 4990, Valid Cost: 0.509, Valid Accuracy: 0.843


100%|██████████| 5000/5000 [43:12<00:00,  1.93it/s]

EPOCH: 5000, Valid Cost: 0.509, Valid Accuracy: 0.843


In [23]:

softmax(np.matmul(x_test, W) + b)
y_pred_test = softmax(np.matmul(x_test, W) + b).argmax(axis=1) # WRITE ME

submission = pd.Series(y_pred_test, name='label')
submission.to_csv('drive/MyDrive/Colab Notebooks/DeepLearning/Lecture02/submission_pred.csv', header=True, index_label='id')

In [14]:
print(y_train)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
